In [1]:
import ipywidgets as widgets
import numpy as np
import keras

from IPython import display as nb
import utils

2024-12-02 09:00:00.103669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 09:00:00.373740: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 09:00:00.432108: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 09:00:00.831496: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-02 09:00:04.169948: W tensorflow/compiler/tf2

In [2]:
packed_model = keras.models.load_model("../model/2024-11-27_00-50-54.keras")
packed_model.layers

I0000 00:00:1733104811.170724    3199 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733104812.178651    3199 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733104812.179168    3199 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733104812.190978    3199 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733104812.191138    3199 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

[<WordPieceTokenizer name=word_piece_tokenizer, built=False>,
 <StringLookup name=string_lookup, built=False>,
 <Sequential name=sequential, built=True>]

In [3]:
tokenizer = packed_model.layers[0]
label_lookup = packed_model.layers[1]

model = packed_model.layers[2]
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 64, 100)        │        26,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64)             │        34,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │            28 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 182,900 (714.46 KB)

 Trainable params: 60,966 (238.15 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 121,934 (476.31 KB)

In [4]:
w_text = widgets.Text(placeholder="Task")
w_text.continuous_update = False
nb.display(w_text)

def on_submit(value):
    nb.clear_output()
    nb.display(w_text)

    task = value["new"].strip()
    print(f"Input: {task}")

    task = utils.lemmatize(task)
    print(f"Lemmatized input: {task}")

    task_vec = tokenizer(task).numpy()
    print(f"Vectorized input: {tokenizer.detokenize(task_vec)}")
    print(f"{task_vec}\n")
    task_vec = np.reshape(task_vec, (1, len(task_vec)))

    prediction = model.predict(task_vec)[0]
    print(f"Prediction vector: {prediction}")

    label_vocab = label_lookup.get_vocabulary()
    for label, probability in zip(label_vocab, prediction):
        percentage = probability * 100
        print(f"\t{label}: {percentage:.5f}%")

    best = label_vocab[np.argmax(prediction)]
    print(f"Prediction: {best}")

w_text.observe(on_submit, "value")


Text(value='', continuous_update=False, placeholder='Task')

In [5]:
model.input_shape

(None, 64)